## Run the code below and finish the questions.

In [1]:
import atoti as tt

Welcome to Atoti 0.8.10!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

Atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, you can request a trial license at https://www.atoti.io/evaluation-license-request.

You can hide this message by setting the `ATOTI_HIDE_EULA_MESSAGE` environment variable to True.


In [3]:
session = tt.Session()

Closing existing "Unnamed" session to create the new one.


In [4]:
# load fact and dimension tables
sales_table = session.read_csv("data/sales.csv", keys=["Sale ID"])
products_table = session.read_csv("data/products.csv", keys=["Product"])
shops_table = session.read_csv("data/shops.csv", keys=["Shop ID"])

In [5]:
# view the product table details
products_table.head()

,Category,Sub category,Size,Purchase price,Color,Brand
Product,,,,,,
BED_6,Furniture,Bed,King,375.0,white,Basic
HOO_14,Cloth,Hoodie,L,39.0,brown,Over
TAB_22,Furniture,Table,2m40,280.0,blue,Basic
TSH_30,Cloth,T-shirt,M,19.0,black,Over
SHO_38,Cloth,Shoes,9,44.0,red,Over


In [6]:
# view the shop table details
shops_table.head()

,City,State or region,Country,Shop size
Shop ID,,,,
shop_6,San Antonio,Texas,USA,small
shop_14,Los Angeles,California,USA,small
shop_22,Marseille,Provence-Alpes-Côte d'Azur,France,small
shop_30,San Francisco,California,USA,medium
shop_38,Saint-Étienne,Auvergne-Rhône-Alpes,France,medium


In [7]:
# view the sale table details
sales_table.head()

,Date,Shop,Product,Quantity,Unit price
Sale ID,,,,,
S000001427,2021-01-18,shop_27,BED_24,1.0,150.0
S000000003,2021-02-01,shop_3,BED_3,1.0,150.0
S000000008,2021-01-27,shop_8,TSH_8,1.0,20.0
S000000012,2021-01-23,shop_12,HOO_12,1.0,45.0
S000000021,2021-01-14,shop_21,TAB_21,1.0,210.0


In [8]:
# star schema
sales_table.join(products_table, sales_table["Product"] == products_table["Product"])
sales_table.join(shops_table, sales_table["Shop"] == shops_table["Shop ID"])
# visualise the schema
session.tables.schema

```mermaid
erDiagram
  "Products" {
    _ String PK "Product"
    _ String "Category"
    _ String "Sub category"
    _ String "Size"
    nullable double "Purchase price"
    _ String "Color"
    _ String "Brand"
  }
  "Sales" {
    _ String PK "Sale ID"
    _ LocalDate "Date"
    _ String "Shop"
    _ String "Product"
    nullable double "Quantity"
    nullable double "Unit price"
  }
  "Shops" {
    _ String PK "Shop ID"
    _ String "City"
    _ String "State or region"
    _ String "Country"
    _ String "Shop size"
  }
  "Sales" }o--o| "Shops" : "`Shop` == `Shop ID`"
  "Sales" }o--o| "Products" : "`Product` == `Product`"
```


In [9]:
# create cube
cube = session.create_cube(sales_table)
cube

In [10]:
# The cube has automatically created a hierarchy, level and measures
hierarchies, levels, measures = cube.hierarchies, cube.levels, cube.measures

In [11]:
measures

{'contributors.COUNT': <atoti.measure.Measure object at 0x7fffb04f6790>, 'update.TIMESTAMP': <atoti.measure.Measure object at 0x7fffb04f6810>, 'Unit price.SUM': <atoti.measure.Measure object at 0x7fffb04f7650>, 'Unit price.MEAN': <atoti.measure.Measure object at 0x7fffb04f7710>, 'Quantity.SUM': <atoti.measure.Measure object at 0x7fffb04f7210>, 'Quantity.MEAN': <atoti.measure.Measure object at 0x7fffb04f7110>}

Some measures have been automatically created from numeric columns but are not useful. For instance, Unit Price.SUM does not really make sense as we never want to sum the unit prices. We can delete it:

In [12]:
del measures["Unit price.SUM"]
measures

{'contributors.COUNT': <atoti.measure.Measure object at 0x7fffb04d1910>, 'update.TIMESTAMP': <atoti.measure.Measure object at 0x7fffb04d1710>, 'Unit price.MEAN': <atoti.measure.Measure object at 0x7fffb04d1a90>, 'Quantity.SUM': <atoti.measure.Measure object at 0x7fffb04d1050>, 'Quantity.MEAN': <atoti.measure.Measure object at 0x7fffb04d20d0>}

you can also create some measures based on the need, e.g. tt.agg.sum() and tt.agg.mean()
```python
m["Amount.SUM"] = tt.agg.sum(sales_table["Quantity"] * sales_table["Unit price"])
m["Amount.MEAN"] = tt.agg.mean(
    sales_table["Quantity"] * sales_table["Unit price"],
)
```

In [13]:
hierarchies

{('Sales', 'Sale ID'): <atoti.hierarchy.Hierarchy object at 0x7fffb17b75d0>, ('Shops', 'Shop size'): <atoti.hierarchy.Hierarchy object at 0x7fffb04e4e90>, ('Shops', 'State or region'): <atoti.hierarchy.Hierarchy object at 0x7fffb050c450>, ('Shops', 'Country'): <atoti.hierarchy.Hierarchy object at 0x7fffb050c610>, ('Products', 'Color'): <atoti.hierarchy.Hierarchy object at 0x7fffb050c750>, ('Sales', 'Shop'): <atoti.hierarchy.Hierarchy object at 0x7fffb050c890>, ('Products', 'Brand'): <atoti.hierarchy.Hierarchy object at 0x7fffb050c9d0>, ('Sales', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fffb050cb10>, ('Products', 'Sub category'): <atoti.hierarchy.Hierarchy object at 0x7fffb050cc50>, ('Products', 'Category'): <atoti.hierarchy.Hierarchy object at 0x7fffb050cd90>, ('Products', 'Size'): <atoti.hierarchy.Hierarchy object at 0x7fffb050ced0>, ('Shops', 'City'): <atoti.hierarchy.Hierarchy object at 0x7fffb050d010>, ('Sales', 'Date'): <atoti.hierarchy.Hierarchy object at 0x7fffb050d150>}

You can see that the cube has automatically created a hierarchy for each non-numeric column in each table.

In [14]:
# Hierarchies are made of levels in Atoti. Levels of the same hierarchy are attributes with a parent-child relationship.
levels

In [15]:
# In Products dimension table, we can group the Category, SubCategory and Product ID levels into a Product hierarchy:

# option 1:
#hierarchies["Product"] = [levels["Product"], levels["Sub category"], levels["Category"]]

# option 2:
hierarchies["Product"] = [levels["Category"], levels["Sub category"], levels["Product"]]

hierarchies
# Your task: Do the above two options have the same results? Why?

{('Sales', 'Sale ID'): <atoti.hierarchy.Hierarchy object at 0x7fffb05252d0>, ('Shops', 'Shop size'): <atoti.hierarchy.Hierarchy object at 0x7fffb0524910>, ('Shops', 'State or region'): <atoti.hierarchy.Hierarchy object at 0x7fffb0526250>, ('Shops', 'Country'): <atoti.hierarchy.Hierarchy object at 0x7fffb04ec310>, ('Products', 'Color'): <atoti.hierarchy.Hierarchy object at 0x7fffb04ec090>, ('Sales', 'Shop'): <atoti.hierarchy.Hierarchy object at 0x7fffb04ec490>, ('Products', 'Brand'): <atoti.hierarchy.Hierarchy object at 0x7fffb04edd10>, ('Sales', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fffb04ef3d0>, ('Products', 'Sub category'): <atoti.hierarchy.Hierarchy object at 0x7fffb04edb90>, ('Products', 'Category'): <atoti.hierarchy.Hierarchy object at 0x7fffb04ef910>, ('Products', 'Size'): <atoti.hierarchy.Hierarchy object at 0x7fffb04f4d90>, ('Shops', 'City'): <atoti.hierarchy.Hierarchy object at 0x7fffb04f5a50>, ('Sales', 'Date'): <atoti.hierarchy.Hierarchy object at 0x7fffb04f5ed0>, ('Products', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fffb04f6e10>}

In [16]:
# remove the old hierarchy
del hierarchies["Category"]
del hierarchies["Sub category"]

In [17]:
hierarchies

{('Products', 'Brand'): <atoti.hierarchy.Hierarchy object at 0x7fffb1746f90>, ('Sales', 'Sale ID'): <atoti.hierarchy.Hierarchy object at 0x7fffb0364250>, ('Shops', 'Shop size'): <atoti.hierarchy.Hierarchy object at 0x7fffb0367f50>, ('Sales', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fffb036c0d0>, ('Products', 'Size'): <atoti.hierarchy.Hierarchy object at 0x7fffb036c210>, ('Shops', 'State or region'): <atoti.hierarchy.Hierarchy object at 0x7fffb036c350>, ('Shops', 'City'): <atoti.hierarchy.Hierarchy object at 0x7fffb036c490>, ('Sales', 'Date'): <atoti.hierarchy.Hierarchy object at 0x7fffb036c5d0>, ('Products', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fffb036c710>, ('Shops', 'Country'): <atoti.hierarchy.Hierarchy object at 0x7fffb036c9d0>, ('Products', 'Color'): <atoti.hierarchy.Hierarchy object at 0x7fffb036cb10>, ('Sales', 'Shop'): <atoti.hierarchy.Hierarchy object at 0x7fffb036cc50>}

In [18]:
# In Shops dimension table, we can group City, State or Region and Country into a Geography hierarchy:

# option 1:
# hierarchies["Geography"] = [
#     shops_table["Country"],
#     shops_table["State or region"],
#     shops_table["City"],
# ]

# option 2:
hierarchies["Geography"] = [levels["Country"], levels["State or region"], levels["City"]]

# Your task: Do the above two options have the same results?

In [19]:
del hierarchies["Country"]
del hierarchies["State or region"]
del hierarchies["City"]

In [20]:
hierarchies

{('Products', 'Brand'): <atoti.hierarchy.Hierarchy object at 0x7fffb035f2d0>, ('Sales', 'Sale ID'): <atoti.hierarchy.Hierarchy object at 0x7fffb035df50>, ('Shops', 'Shop size'): <atoti.hierarchy.Hierarchy object at 0x7fffb035d350>, ('Sales', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fffb0342810>, ('Products', 'Size'): <atoti.hierarchy.Hierarchy object at 0x7fffb0343cd0>, ('Sales', 'Date'): <atoti.hierarchy.Hierarchy object at 0x7fffb0343350>, ('Products', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fffb03422d0>, ('Products', 'Color'): <atoti.hierarchy.Hierarchy object at 0x7fffb0342c50>, ('Sales', 'Shop'): <atoti.hierarchy.Hierarchy object at 0x7fffb0343050>, ('Shops', 'Geography'): <atoti.hierarchy.Hierarchy object at 0x7fffb03427d0>}

## After cleaning all hierarchies, we can query our data warehouse. Think about the starnet!

In [21]:
# query 1 (roll-up to ALL): what is the sold quantities over the whole dataset?
cube.query(measures["Quantity.SUM"])

,Quantity.SUM
0,"8,077.00"


In [22]:
levels

In [23]:
# query 2 (dice): what is the sold quantities for each shop?
cube.query(measures["Quantity.SUM"], levels=[levels["Shop"]])

# Your task: Add "Shop" to the Geography hierarchy and observe the outcome.
# Hint: If errors occur in the query after adding "Shop" to the Geography hierarchy, check for duplicate "Shop" levels.
# If there are levels with the same name but different parents, specify the parent, e.g., cube.query(measures["Quantity.SUM"], levels=[levels[('Shops', 'Geography', 'Shop')]])

,Quantity.SUM
Shop,
shop_0,202.00
shop_1,202.00
shop_10,203.00
shop_11,203.00
shop_12,201.00
shop_13,202.00
shop_14,202.00
shop_15,202.00
shop_16,201.00


In [24]:
# query 3 (slice): what is the sold quantities for shop_36?
cube.query(
    measures["Quantity.SUM"],
    filter=levels["Shop"] == "shop_36"
)

# Your task: after adding "Shop" to the Geography hierarchy, you need to specify the parent in filter. 
# e.g. filter=levels[('Sales', 'Shop', 'Shop')]== "shop_36"

,Quantity.SUM
0,203.00


In [25]:
# query 4: What is the sold quantities sold by date and product?
cube.query(measures["Quantity.SUM"], levels=[levels["Date"], levels[('Sales', 'Product', 'Product')]])

# Your task: run the following code and explain why it does not work. Hint: check how many 'Product' in levels
# cube.query(measures["Quantity.SUM"], levels=[levels["Date"], levels["Product"]])

Quantity.SUM
Date       Product             
2021-01-06 BED_24          8.00
           BED_25          4.00
           BED_26          6.00
           BED_27          4.00
           BED_3           2.00
...                         ...
2021-02-04 TSH_52          6.00
           TSH_53          4.00
           TSH_7           3.00
           TSH_8           5.00
           TSH_9           3.00

[1830 rows x 1 columns]

In [26]:
cube.query(measures["Quantity.SUM"], levels=[levels["Date"], levels[('Products', 'Product', 'Sub category')]])

Quantity.SUM
Date       Category  Sub category             
2021-01-06 Cloth     Hoodie              37.00
                     Shoes               71.00
                     T-shirt             66.00
           Furniture Bed                 53.00
                     Chair               15.00
...                                        ...
2021-02-04 Cloth     Shoes               76.00
                     T-shirt             69.00
           Furniture Bed                 49.00
                     Chair               13.00
                     Table               26.00

[180 rows x 1 columns]

In [27]:
# rollup
cube.query(measures["Quantity.SUM"], levels=[levels["Date"], levels[('Products', 'Product', 'Category')]])

Quantity.SUM
Date       Category              
2021-01-06 Cloth           174.00
           Furniture        94.00
2021-01-07 Cloth           177.00
           Furniture        89.00
2021-01-08 Cloth           174.00
           Furniture        94.00
2021-01-09 Cloth           177.00
           Furniture        93.00
2021-01-10 Cloth           177.00
           Furniture        91.00
2021-01-11 Cloth           176.00
           Furniture        93.00
2021-01-12 Cloth           177.00
           Furniture        88.00
2021-01-13 Cloth           178.00
           Furniture        92.00
2021-01-14 Cloth           175.00
           Furniture        93.00
2021-01-15 Cloth           179.00
           Furniture        91.00
2021-01-16 Cloth           178.00
           Furniture        89.00
2021-01-17 Cloth           178.00
           Furniture        93.00
2021-01-18 Cloth           180.00
           Furniture        91.00
2021-01-19 Cloth           181.00
           Furniture        90.00
2021-01-20 Cloth           176.00
           Furniture        91.00
2021-01-21 Cloth           178.00
           Furniture        92.00
2021-01-22 Cloth           179.00
           Furniture        92.00
2021-01-23 Cloth           175.00
           Furniture        94.00
2021-01-24 Cloth           183.00
           Furniture        87.00
2021-01-25 Cloth           176.00
           Furniture        93.00
2021-01-26 Cloth           184.00
           Furniture        87.00
2021-01-27 Cloth           173.00
           Furniture        96.00
2021-01-28 Cloth           179.00
           Furniture        92.00
2021-01-29 Cloth           168.00
           Furniture        99.00
2021-01-30 Cloth           172.00
           Furniture        99.00
2021-01-31 Cloth           173.00
           Furniture        98.00
2021-02-01 Cloth           170.00
           Furniture       101.00
2021-02-02 Cloth           176.00
           Furniture        91.00
2021-02-03 Cloth           174.00
           Furniture        96.00
2021-02-04 Cloth           183.00
           Furniture        88.00

In [28]:
# drilldown
cube.query(measures["Quantity.SUM"], levels=[levels["Date"], levels[('Products', 'Product', 'Product')]])

Quantity.SUM
Date       Category  Sub category Product             
2021-01-06 Cloth     Hoodie       HOO_12          4.00
                                  HOO_13          6.00
                                  HOO_14          4.00
                                  HOO_33          5.00
                                  HOO_34          2.00
...                                                ...
2021-02-04 Furniture Table        TAB_1           3.00
                                  TAB_21          6.00
                                  TAB_22          3.00
                                  TAB_42          4.00
                                  TAB_43          6.00

[1830 rows x 1 columns]

In [29]:
# query 5: What is the sold quantities sold by date and product for shop_36
cube.query(
    measures["Quantity.SUM"],
    levels=[levels["Date"], levels[('Sales', 'Product', 'Product')]],
    filter=levels["Shop"] == "shop_36",
)

Quantity.SUM
Date       Product             
2021-01-09 BED_25          1.00
           BED_27          2.00
           BED_3           1.00
           BED_45          1.00
           BED_46          1.00
...                         ...
2021-01-29 TSH_32          1.00
           TSH_49          3.00
           TSH_51          1.00
           TSH_53          4.00
           TSH_8           1.00

[125 rows x 1 columns]

In [30]:
# query
session.link

http://localhost:41079

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.